In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
pwd

'/content'

In [0]:
root_path = 'gdrive/My Drive/capstone_project/input/recognizing-faces-in-the-wild'

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir(root_path))

# Any results you write to the current directory are saved as output.

['test', 'train', '.ipynb_checkpoints', 'kernel_submission.csv', 'Data Cleaning and Preprocessing.ipynb', '.DS_Store', 'sample_submission.csv', 'combinations.txt', 'Untitled.ipynb', 'train_relationships.csv', 'test.zip', 'train.zip', 'df_final.csv', 'kinship_detection_with_vgg16.ipynb', 'open-kimono-kinship-notebook-0-792lb.ipynb', 'kinship-detection-with-vgg16.ipynb']


Necassary Imports

In [0]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-zi3msh1v
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-zi3msh1v
  Stored in directory: /tmp/pip-ephem-wheel-cache-f83fmjik/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [0]:
import h5py
from collections import defaultdict
from glob import glob
from random import choice, sample

import cv2
import numpy as np
import pandas as pd
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Input, Dense, Flatten, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, Add, Conv2D
from keras.models import Model
from keras.optimizers import Adam
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace

In [0]:
train_file_path = root_path + "/train_relationships.csv"
train_folders_path = root_path + "/train/"
val_famillies = "F09"
train_folders_path

'gdrive/My Drive/capstone_project/input/recognizing-faces-in-the-wild/train/'

In [0]:
train_file_path

'gdrive/My Drive/capstone_project/input/recognizing-faces-in-the-wild/train_relationships.csv'

In [0]:
train_folders_path

'gdrive/My Drive/capstone_project/input/recognizing-faces-in-the-wild/train/'

In [0]:
all_images = glob(root_path + "/train/" + "*/*/*.jpg")
len(all_images)

12429

Validation set consists only family 9

In [0]:
train_images = [x for x in all_images if val_famillies not in x]
val_images = [x for x in all_images if val_famillies in x]

In [0]:
train_person_to_images_map = defaultdict(list)

In [0]:
ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]

In [0]:
for x in train_images:
    train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

val_person_to_images_map = defaultdict(list)

for x in val_images:
    val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

relationships = pd.read_csv(train_file_path)
relationships = list(zip(relationships.p1.values, relationships.p2.values))
relationships = [x for x in relationships if x[0] in ppl and x[1] in ppl]

train = [x for x in relationships if val_famillies not in x[0]]
val = [x for x in relationships if val_famillies in x[0]]

In [0]:
def read_img(path):
    img = cv2.imread(path)
    img = np.array(img).astype(np.float)
    return preprocess_input(img, version=2)


def gen(list_tuples, person_to_images_map, batch_size=16):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size // 2)
        labels = [1] * len(batch_tuples)
        while len(batch_tuples) < batch_size:
            p1 = choice(ppl)
            p2 = choice(ppl)

            if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
                batch_tuples.append((p1, p2))
                labels.append(0)

        for x in batch_tuples:
            if not len(person_to_images_map[x[0]]):
                print(x[0])

        X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
        X1 = np.array([read_img(x) for x in X1])

        X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
        X2 = np.array([read_img(x) for x in X2])

        yield [X1, X2], labels


def baseline_model():
    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    base_model = VGGFace(model='resnet50', include_top=False)

    for layer in base_model.layers[:-3]:
        layer.trainable = True

    x1 = base_model(input_1)
    x2 = base_model(input_2)


    merged_add = Add()([x1, x2])
    merged_sub = Subtract()([x1,x2])
    
    merged_add = Conv2D(100 , [1,1] )(merged_add)
    merged_sub = Conv2D(100 , [1,1] )(merged_sub)
    
    merged = Concatenate(axis=-1)([merged_add, merged_sub])
    
    merged = Flatten()(merged)

    merged = Dense(100, activation="relu")(merged)
    merged = Dropout(0.3)(merged)
    merged = Dense(25, activation="relu")(merged)
    merged = Dropout(0.3)(merged)
    out = Dense(1, activation="sigmoid")(merged)

    model = Model([input_1, input_2], out)

    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))

    model.summary()

    return model


file_path = "vgg_face.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.1, patience=20, verbose=1)

callbacks_list = [checkpoint, reduce_on_plateau]

model = baseline_model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
vggface_resnet50 (Model)        multiple             23561152    input_4[0][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
add_34 (Add)                    (None, 1, 1, 2048)   0           vggface_resnet50[1][0]           
          

In [0]:
model.fit_generator(gen(train, train_person_to_images_map, batch_size=16), use_multiprocessing=True,
                    validation_data=gen(val, val_person_to_images_map, batch_size=16), epochs=10, verbose=1,
                    workers = 4, callbacks=callbacks_list, steps_per_epoch=200, validation_steps=100)

/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/10
200/200 [==============================] - 229s 1s/step - loss: 1.2658 - acc: 0.5616 - val_loss: 0.7927 - val_acc: 0.6231

Epoch 00001: val_acc improved from -inf to 0.62313, saving model to vgg_face.h5
Epoch 2/10
200/200 [==============================] - 204s 1s/step - loss: 0.8433 - acc: 0.5816 - val_loss: 0.6775 - val_acc: 0.6169

Epoch 00002: val_acc did not improve from 0.62313
Epoch 3/10
200/200 [==============================] - 203s 1s/step - loss: 0.7380 - acc: 0.5931 - val_loss: 0.6549 - val_acc: 0.6331

Epoch 00003: val_acc improved from 0.62313 to 0.63313, saving model to vgg_face.h5
Epoch 4/10
200/200 [==============================] - 204s 1s/step - loss: 0.6906 - acc: 0.6069 - val_loss: 0.6341 - val_acc: 0.6362

Epoch 00004: val_acc improved from 0.63313 to 0.63625, saving model to vgg_face.h5
Epoch 5/10
200/200 [==============================] - 204s 1s/step - loss: 0.6575 - acc: 0.6266 - val_loss: 0.6303 - val_acc: 0.6450

Epoch 00005: val_acc improved from

In [0]:
test_path = root_path + "/test/"


def chunker(seq, size=32):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))


from tqdm import tqdm

submission = pd.read_csv(root_path + '/sample_submission.csv')

predictions = []

for batch in tqdm(chunker(submission.img_pair.values)):
    X1 = [x.split("-")[0] for x in batch]
    X1 = np.array([read_img(test_path + x) for x in X1])

    X2 = [x.split("-")[1] for x in batch]
    X2 = np.array([read_img(test_path + x) for x in X2])

    pred = model.predict([X1, X2]).ravel().tolist()
    predictions += pred

submission['is_related'] = predictions

submission.to_csv("vgg_face.csv", index=False)